# 2. Data Cleaning and Exploratory Data Analysis

In this stage we will look at the data and understand the data quality and quantity.\
This stage will give us an indication of the necessary changes that our data needs before processing to different stages.\

This stages requires the data present in the data directory. \
You will need to download the data using the module in the feature pipeline: "energy_consumption_forecasting\feature_pipeline\etl_data_extraction.py" \
or you can run the data extraction notebook: "notebooks\01_data_extraction.ipynb".

Once data is download you can follow along.